In [ ]:
# Script for the plotting of clustered single cell RNA seq data
# By Louise Baldwin, based on scripts from Sergio Erdal Irac
# takes clustered merged h5ad as input. 

###################
# Set up
###################

# import packages
import numpy as np
import pandas as pd
import scanpy as sc
import os
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.pyplot import rc_context

# directories
os.chdir("/share/ScratchGeneral/loubal/projects/MSC/mouse-single-cell")
in_file = ("data/processed/merged_withscrub.h5ad")
results_file = ("data/processed/clustered_merged_umap.h5ad")
figdir = ("outs/3_umap/figures/")
tabdir = ("outs/3_umap/tables/")
os.makedirs(figdir, exist_ok=True)
os.makedirs(tabdir, exist_ok=True)

# set parameters for scanpy
# verbosity: errors (0), warnings (1), info (2), hints (3), detailed traceback (4)
# change default figdir to desired figdir
sc.settings.verbosity = 2            
sc.logging.print_header()
sc.settings.set_figure_params(dpi=150, facecolor='white')
#sc.set_figure_params(facecolor='white', color_map="viridis")
#sc.settings.figdir='/share/ScratchGeneral/loubal/projects/MSC/mouse-single-cell/outs/QC/figures/'
sc.settings.figdir=figdir


In [ ]:
# read in file
adata=sc.read_h5ad(in_file)
adata

In [ ]:
adata.uns['log1p']["base"] = None

In [ ]:
sc.pl.umap(adata, color=['leiden','ReactionID','Treatment','Tissue','Model'], size=1, 
 show=True, add_outline=True,  frameon=False, ncols=2, wspace=0.5, title='leiden_0.1', save='umap.png')
 

In [ ]:
sc.pl.umap(adata, color="leiden", frameon=False, legend_loc="on data")

In [ ]:
#change resolution to 0.3
sc.tl.leiden(adata, resolution=0.3, key_added='leiden_0.3', random_state=10)

In [ ]:
#plot new resolution clustering
sc.pl.umap(adata, color=['leiden_0.3'], size=1,
add_outline=True,  frameon=False, ncols=2, wspace=0.5, title='leiden_0.3', save="_leiden_03.pdf")

In [ ]:
#calculate and plot counts per cluster, then remove clusters with less than 50 cells
# figure out how to change x and y axis labels to percentage and leiden cluster
counts = adata.obs.groupby(['leiden_0.3']).count().reset_index()
plt.figure(figsize = (20,3))
ax = sns.barplot(x = 'leiden_0.3', y = 'n_counts', data = counts)
ax.set_yscale("log")
plt.savefig((figdir+"cells_per_leiden03_cluster.png"))

In [ ]:
adata._inplace_subset_obs( adata.obs.groupby('leiden_0.3').filter( lambda x : len(x)>50).index)

In [ ]:
counts = adata.obs.groupby(['leiden_0.3']).count().reset_index()
plt.figure(figsize = (20,3))
ax = sns.barplot(x = 'leiden_0.3', y = 'n_counts', data = counts)
ax.set_yscale("log")
plt.savefig(("cells_per_leiden_0.3_cluster_afterfiltering.png"))

In [ ]:
sc.pl.umap(adata, color="leiden_0.3", frameon=False)

In [ ]:
sc.pl.umap(adata, color=['Treatment'], size=1,
 show=False, add_outline=True,  frameon=False, ncols=2, wspace=0.5, title='Treatment')

In [ ]:
sc.tl.rank_genes_groups(adata, groupby='leiden_0.3', key_added='rank_genes_0.3')

In [ ]:
results = adata.uns['rank_genes_0.3']
results['names'].dtype.names

In [ ]:
out = np.array([[0,0,0,0,0]])

In [ ]:
#save genes as csv
for group in results['names'].dtype.names:
    out = np.vstack((out, np.vstack((results['names'][group],
                                     results['scores'][group],
                                     results['pvals_adj'][group],
                                     results['logfoldchanges'][group],
                                     np.array([group] * len(results['names'][group])).astype('object'))).T))

In [ ]:
pd.DataFrame(out).to_csv(tabdir+'leiden_0.3_rank_genes_groups.csv')

In [ ]:
out.shape

In [ ]:
markers = pd.DataFrame(out[1:], columns = ['Gene', 'scores', 'pval_adj', 'lfc', 'cluster'])

In [ ]:
pd.DataFrame.head(markers)

In [ ]:
markers = markers[(markers.pval_adj < 0.05) & (markers.lfc > .5)]

In [ ]:
pd.DataFrame.head(markers)

In [ ]:
markers.to_csv(tabdir+'leiden_0.3_markers_rank_genes_groups.csv')

In [ ]:
results_file = ("data/processed/clustered_merged_umap.h5ad")
adata.write_h5ad(results_file)

In [ ]:
# print some metrics
print(adata.obs['Treatment'].value_counts())
print(adata.obs['Model'].value_counts())
print(adata.obs['Tissue'].value_counts())
print(adata.obs['ReactionID'].value_counts())


In [ ]:
# generate cells per variable and write to df

treatment_stats = adata.obs['Treatment'].value_counts().to_frame()
model_stats = adata.obs['Model'].value_counts().to_frame()
tissue_stats = adata.obs['Tissue'].value_counts().to_frame()
reaction_stats = adata.obs['ReactionID'].value_counts().to_frame()

In [ ]:
# save df as csv
treatment_stats.to_csv((tabdir+'cells_per_treatment.csv'))
model_stats.to_csv(tabdir+'cells_per_model.csv')
tissue_stats.to_csv(tabdir+'cells_per_tissue.csv')
reaction_stats.to_csv(tabdir+'cells_per_reaction.csv')

In [ ]:
type(treatment_stats)

In [ ]:
#need to make sure 4T1 primary tumour control igG is more than 0 - 12490 specificaly
count_again=adata.obs.groupby(['Model', 'Tissue', 'Treatment']).size()
other_df=count_again.to_frame(name='count').reset_index()
other_df

In [ ]:
# split by model

from matplotlib.pyplot import rc_context
def cluster_small_multiples(adata, ReactionID, size=60, frameon=False, legend_loc=None, **kwargs):
    tmp = adata.copy()
    for i,clust in enumerate(adata.obs[ReactionID].cat.categories):
        tmp.obs[clust] = adata.obs[ReactionID].isin([clust]).astype('category')
        tmp.uns[clust+'_colors'] = ['#d3d3d3', adata.uns[ReactionID+'_colors'][i]]
    sc.pl.umap(tmp, groups=tmp.obs[clust].cat.categories[1:].values, color=adata.obs[ReactionID].cat.categories.tolist(), size=5, ncols=2, frameon=False, legend_loc=legend_loc, **kwargs)


with rc_context({'figure.figsize': (3, 2.5)}):
  cluster_small_multiples(adata, 'ReactionID')

In [ ]:
#split by model
def cluster_small_multiples(adata, Model, size=60, frameon=False, legend_loc=None, **kwargs):
    tmp = adata.copy()
    for i,clust in enumerate(adata.obs[Model].cat.categories):
        tmp.obs[clust] = adata.obs[Model].isin([clust]).astype('category')
        tmp.uns[clust+'_colors'] = ['#d3d3d3', adata.uns[Model+'_colors'][i]]
    sc.pl.umap(tmp, groups=tmp.obs[clust].cat.categories[1:].values, color=adata.obs[Model].cat.categories.tolist(), size=5, ncols=2, frameon=False, legend_loc=legend_loc, **kwargs)


with rc_context({'figure.figsize': (3, 2.5)}):
  cluster_small_multiples(adata, 'Model')